In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import FeatureHasher

def label_encode_dataframe(data,columns=data.columns,target=None):
    
    try:
        data = data[columns]
        data.drop([target],inplace=True ,axis=1)
    except :
        logger.error(f'Problem with label encoding : {e})
    train=pd.DataFrame()
    label=LabelEncoder()
    for c in  X_train.columns:
        if(X[c].dtype=='object'):
            train[c]=label.fit_transform(X_train[c])
        else:
            train[c]=X_train[c]
    return train        
        


In [ ]:
#Cyclic Feature
def cyclic_features(data,columns=data.columns,target=None):

    
    try:
        X_train_cyclic=data[columns]
        data.drop([target],inplace=True ,axis=1)
    except Exception as e:
        logger.error(f'Problem with cyclic feature building {e})
    for col in columns:
        X_train_cyclic[col+'_sin']=np.sin((2*np.pi*X_train_cyclic[col])/max(X_train_cyclic[col]))
        X_train_cyclic[col+'_cos']=np.cos((2*np.pi*X_train_cyclic[col])/max(X_train_cyclic[col]))
    X_train_cyclic=X_train_cyclic.drop(columns,axis=1)
    return X_train_cyclic                 
                     


In [ ]:
#hash feature
def hashed_features(data,columns=data.columns,target=None):
    try:
        X_train_hash=data[columns]
        data.drop([target],inplace=True ,axis=1)
    except Exception as e:
        logger.error(f'Problem with hashed feature building {e})
    for c in X.columns:
        X_train_hash[c]=X[c].astype('str')  
    hashing=FeatureHasher(input_type='string')
    hashed=hashing.transform(X_train_hash.values)    
    return hashed                
    
    

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one=OneHotEncoder()

one.fit(X)
train=one.transform(X)

print('train data set has got {} rows and {} columns'.format(train.shape[0],train.shape[1]))

In [ ]:
X_train_stat=X.copy()
for c in X_train_stat.columns:
    if(X_train_stat[c].dtype=='object'):
        X_train_stat[c]=X_train_stat[c].astype('category')
        counts=X_train_stat[c].value_counts()
        counts=counts.sort_index()
        counts=counts.fillna(0)
        counts += np.random.rand(len(counts))/1000
        X_train_stat[c].cat.categories=counts

In [ ]:
def encode_onehot(train, test, categ_variables):
    df_onehot = pd.get_dummies(pd.concat([train[cat_vars], test[cat_vars]]).astype(str))
    return df_onehot[:len(train)], df_onehot[len(train):]



In [ ]:
def encode_target_smooth(data, target, categ_variables, smooth):
    """    
    Apply target encoding with smoothing.
    
    Parameters
    ----------
    data: pd.DataFrame
    target: str, dependent variable
    categ_variables: list of str, variables to encode
    smooth: int, number of observations to weigh global average with
    
    Returns
    --------
    encoded_dataset: pd.DataFrame
    code_map: dict, mapping to be used on validation/test datasets 
    defaul_map: dict, mapping to replace previously unseen values with
    """
    train_target = data.copy()
    code_map = dict()    # stores mapping between original and encoded values
    default_map = dict() # stores global average of each variable
    
    for v in categ_variables:
        prior = data[target].mean()
        n = data.groupby(v).size()
        mu = data.groupby(v)[target].mean()
        mu_smoothed = (n * mu + smooth * prior) / (n + smooth)
        
        train_target.loc[:, v] = train_target[v].map(mu_smoothed)        
        code_map[v] = mu_smoothed
        default_map[v] = prior        
    return train_target, code_map, default_map


#invocation
train_target_smooth, target_map, default_map = encode_target_smooth(train, 'adoptionspeed', cat_vars, 500)
test_target_smooth = test.copy()
for v in cat_vars:
    test_target_smooth.loc[:, v] = test_target_smooth[v].map(target_map[v])


In [ ]:
#https://www.kaggle.com/dustinthewind/making-sense-of-mean-encoding

def impact_coding_leak(data, feature, target, n_folds=20, n_inner_folds=10):
    from sklearn.model_selection import StratifiedKFold
    '''
    ! Using oof_default_mean for encoding inner folds introduces leak.
    
    Source: https://www.kaggle.com/tnarik/likelihood-encoding-of-categorical-features
    
    Changelog:    
    a) Replaced KFold with StratifiedFold due to class imbalance
    b) Rewrote .apply() with .map() for readability
    c) Removed redundant apply in the inner loop
    '''
    impact_coded = pd.Series()
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True) # KFold in the original
    oof_mean_cv = pd.DataFrame()
    split = 0
    for infold, oof in kf.split(data[feature], data[target]):

        kf_inner = StratifiedKFold(n_splits=n_inner_folds, shuffle=True)
        inner_split = 0
        inner_oof_mean_cv = pd.DataFrame()
        oof_default_inner_mean = data.iloc[infold][target].mean()
        
        for infold_inner, oof_inner in kf_inner.split(data.iloc[infold], data.loc[infold, target]):
            # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
            oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()

            # Also populate mapping (this has all group -> mean for all inner CV folds)
            inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
            inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
            inner_split += 1

        # compute mean for each value of categorical value across oof iterations
        inner_oof_mean_cv_map = inner_oof_mean_cv.mean(axis=1)

        # Also populate mapping
        oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
        oof_mean_cv.fillna(value=oof_default_mean, inplace=True)
        split += 1

        feature_mean = data.loc[oof, feature].map(inner_oof_mean_cv_map).fillna(oof_default_mean)
        impact_coded = impact_coded.append(feature_mean)
            
    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean


def impact_coding(data, feature, target, n_folds=20, n_inner_folds=10):
    from sklearn.model_selection import StratifiedKFold
    '''
    ! Using oof_default_mean for encoding inner folds introduces leak.
    
    Changelog:    
    a) Replaced KFold with StratifiedFold due to class imbalance
    b) Rewrote .apply() with .map() for readability
    c) Removed redundant apply in the inner loop
    d) Removed global average; use local mean to fill NaN values in out-of-fold set
    '''
    impact_coded = pd.Series()
        
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True) # KFold in the original
    oof_mean_cv = pd.DataFrame()
    split = 0
    for infold, oof in kf.split(data[feature], data[target]):

        kf_inner = StratifiedKFold(n_splits=n_inner_folds, shuffle=True)
        inner_split = 0
        inner_oof_mean_cv = pd.DataFrame()
        oof_default_inner_mean = data.iloc[infold][target].mean()
        
        for infold_inner, oof_inner in kf_inner.split(data.iloc[infold], data.loc[infold, target]):
                    
            # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
            oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
            
            # Also populate mapping (this has all group -> mean for all inner CV folds)
            inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
            inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
            inner_split += 1

        # compute mean for each value of categorical value across oof iterations
        inner_oof_mean_cv_map = inner_oof_mean_cv.mean(axis=1)

        # Also populate mapping
        oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
        oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True) # <- local mean as default
        split += 1

        feature_mean = data.loc[oof, feature].map(inner_oof_mean_cv_map).fillna(oof_default_inner_mean)
        impact_coded = impact_coded.append(feature_mean)
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean

def encode_target_cv(data, target, categ_variables, impact_coder=impact_coding):
    """Apply original function for each <categ_variables> in  <data>
    Reduced number of validation folds
    """
    train_target = data.copy() 
    
    code_map = dict()
    default_map = dict()
    for f in categ_variables:
        train_target.loc[:, f], code_map[f], default_map[f] = impact_coder(train_target, f, target)
        
    return train_target, code_map, default_map




In [ ]:
def cross_validate_encoder(X, target, categorical_vars, encoder,
                           model, n_splits=10, **kwargs):
    """Evaluate perfomance of encoding categorical varaibles with <encoder> by fitting 
    <model> and measuring average kappa on <n_samples> cross validation.
    
    Make sure to apply mean encoding only to infold set.
            
    Parameters
    ----------
    X: pd.DataFrame, train data
    target: str, response variable
    categorical_vars: list of str, categorical variables to encode
    encoder: custom function to apply
    model: sklearn model to fit
    n_splits: number of cross-validation folds
    **kwargs: key-word arguments to encoder
    
    Returns
    ----------
    metric_cvs: np.array of float, metrics computed on the held-out fold
    """     
    skf = StratifiedKFold(n_splits=n_splits, random_state=20190301)
    metric_cvs = list()
    
    for fold_idx, val_idx in skf.split(X=X, y=X[target]):
        train_fold, valid_fold = X.loc[fold_idx].reset_index(drop=True), \
                                 X.loc[val_idx].reset_index(drop=True)
        
        # apply encoding to k-th fold and validation set 
        train_fold, code_map, default_map = encoder(train_fold, target, categorical_vars, **kwargs)
        for v in categorical_vars:
            valid_fold.loc[:, v] = valid_fold[v].map(code_map[v]).fillna(default_map[v])
        
        # fit model on training fold
        model.fit(train_fold[categorical_vars], train_fold[target])
        # predict out-of-fold
        oof_pred = model.predict(valid_fold[categorical_vars])
        metric_cvs.append(cohen_kappa_score(valid_fold[target], oof_pred, weights='quadratic'))        
    return np.array(metric_cvs)

In [ ]:
%%time

X_target=df_train.copy()
X_target['day']=X_target['day'].astype('object')
X_target['month']=X_target['month'].astype('object')
for col in X_target.columns:
    if (X_target[col].dtype=='object'):
        target= dict ( X_target.groupby(col)['target'].agg('sum')/X_target.groupby(col)['target'].agg('count'))
        X_target[col]=X_target[col].replace(target).values
        
    

# DateTime Operations

In [ ]:
# Quick Extractions
df['only_date'] = df['CreationDate'].dt.date
df['time'] = df['CreationDate'].dt.time
df['year'] = df['CreationDate'].dt.year.astype('Int16')
df['month'] = df['CreationDate'].dt.month.astype('Int8')
df['month_name'] = df['CreationDate'].dt.strftime('%B') 
df['day'] = df['CreationDate'].dt.day.astype('Int8')
df['weekday'] = df['CreationDate'].dt.strftime('%A')  
df['week_num'] = df['CreationDate'].dt.strftime('%W') #  (Monday as first day)


In [ ]:
#Quick Difference 
def months(d1,d2):
    d1=pd.to_datetime(d1)
    d2=pd.to_datetime(d2)
    return (d1.year-d2.year)*12 + (d1.month-d2.month) 
df['diff_year']=round((pd.to_datetime(df['future_date'])-pd.to_datetime(df['CreationDate'])).dt.days/365).astype(int)
df['diff_month']=df.apply(lambda x:months(x.future_date,x.CreationDate),axis=1)
df['diff_days']=(pd.to_datetime(df['future_date'])-pd.to_datetime(df['CreationDate'])).dt.days
df['diff_hours']=(df['diff_days']*24)+(((pd.to_datetime(df['future_date'])-pd.to_datetime(df['CreationDate'])).dt.seconds)// 3600).astype(int)
df['diff_minutes']=(df['diff_days']*24*60)+(((pd.to_datetime(df['future_date'])-pd.to_datetime(df['CreationDate'])).dt.seconds)//60).astype(int)


In [ ]:
#Quick Sum
df['add_days']=df['CreationDate']+timedelta(5)
df['add_hours_min']=df['CreationDate']+timedelta(hours=5,minutes=30)

In [ ]:
#
def weekend(date):
    date=pd.to_datetime(date)
    if((date.strftime('%w')=='0')|(date.strftime('%w')=='6')):
        return "weekend"
    else:
        return "working day"
    
df['weekend_or_not']=df['CreationDate'].apply(lambda x:weekend(x))   

def leap(date):
    date=pd.to_datetime(date)
    if(date.year%4==0):
        return "leap"
    else:
        return "non leap"
    
df['leap']=df['CreationDate'].apply(lambda x:leap(x))


In [ ]:
#Days difference from next row(Shift)
shift_df['diff_from_last_date']=(shift_df['CreationDate']-shift_df['CreationDate'].shift()).dt.days
